In [1]:
## QICK LIBRARY
from qick import *

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger, LFSR

import numpy as np
import time

import qick.tprocv2_assembler as qasm
from importlib import reload  # Python 3.4+
    
##### Load FGPA BitStream
soc = QickSoc('./qick_4x2_jac.bit')


In [17]:
reload(qasm)
asm = """
//TEST program//TEST program
.ALIAS repeat_cnt r0
.ALIAS pulse_start  r1
.ALIAS pulse_moving_off   r3

//.CONST pulse_repeat_w     #500000
//.CONST pulse_w            #1000
//.CONST pulse_central_off  #500
//.CONST total_repeat #500
//.CONST velocity #2

.CONST pulse_repeat_w     #500
.CONST pulse_w            #100
.CONST pulse_central_off  #50
.CONST total_repeat #10
.CONST velocity #2


REG_WR s12 imm #12
REG_WR s13 imm #13

INIT:

REG_WR repeat_cnt imm total_repeat
REG_WR repeat_cnt op -op(repeat_cnt - #1) -uf

// set up pulse wave
REG_WR w_freq imm #5000000
REG_WR w_phase imm #700000000
REG_WR w_gain imm #50000
REG_WR w_length imm #215

TRIG_LOOP:
   TIME inc_ref pulse_repeat_w
   REG_WR pulse_start imm #1000
   REG_WR out_usr_time op -op(pulse_start)
   // ENVELOPE RISE
   TRIG p0 set
   WPORT_WR p0 r_wave
      // CENTRAL PULSE
      REG_WR out_usr_time op -op(pulse_start + pulse_central_off)
      TRIG p1 set 
      REG_WR out_usr_time op -op(out_usr_time + #10)
      TRIG p1 clr 
      // MOVING PULSE
      REG_WR pulse_moving_off op -op(pulse_moving_off + velocity)
      REG_WR out_usr_time op -op(pulse_start + pulse_moving_off)
      TRIG p2 set
      REG_WR out_usr_time op -op(out_usr_time + #10)
      TRIG p2 clr
      // ENVELOPE FALL
      REG_WR out_usr_time op -op(pulse_start + pulse_w) 
      TRIG p0 clr 
   //JUMP TRIG_LOOP -wr(repeat_cnt op) -op(repeat_cnt - #1) -if(NZ) -uf
   WAIT time @100
   REG_WR repeat_cnt op -op(repeat_cnt - #1) -uf
   JUMP TRIG_LOOP -if(NZ)

END: 
    JUMP END

"""

p_txt, p_bin             = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)

# Python READS CORE0_W_DT
soc.tproc.read_sel  = 1
soc.tproc.reset()

print('TIME_USR: ', soc.tproc.time_usr)
QTY = 200


print('--- LOADING MEM')
soc.tproc.Load_PMEM(p_bin)
#The ZCU216 has a tproc_start and tproc_stop
soc.tproc.start()
#soc.tproc.axi()



{'P_ADDR': 25, 'LINE': 55, 'TIME': '@100', 'CMD': 'WAIT', 'C_OP': 'time', 'UF': '0'}
TIME_USR:  0
--- LOADING MEM
